In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from tensorflow.keras import Sequential, layers
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", False)
from sklearn.metrics import mean_absolute_percentage_error
from datetime import date, timedelta, datetime
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam

# Prepare dataset

## Load data

In [14]:
df = pd.read_csv('data/data_preparation.csv', index_col=[0])

In [15]:
df

,Date,Code INSEE région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,TCO Thermique (%),TCH Thermique (%),TCO Nucléaire (%),TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),Column 30,YEAR,MONTH,DAY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,QV2M,RH2M,PRECTOTCORR,PS,WS10M,WS50M,...,bank_holiday,school_holidays,season,num_day,sin365_1,cos365_1,sin365_2,cos365_2,sin365_3,cos365_3,t - 1,t - 2,t - 3,t - 4,t - 5,t - 6,t - 7,t - 8,t - 9,t - 10,t - 11,t - 12,t - 13,t - 14,t - 15,t - 16,t - 17,t - 18,t - 19,t - 20,t - 21,t - 22,t - 23,t - 24,t - 25,t - 26,t - 27,t - 28,t - 29,t - 30
32,2013-02-02,4464,262321.0,15464.0,0.0,1574.0,2710.0,57188.0,0.0,3215.0,182068.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,2,2,5.55,2.13,3.84,5.69,7.38,8.87,1.48,4.82,79.19,1.94,95.99,12.23,15.88,...,0.0,0,1,33,0.537677,0.843151,0.906686,0.421806,0.764891,-0.644159,266134.0,268874.0,285681.0,303496.0,303546.0,298866.0,309993.0,321288.0,313799.0,314414.0,306209.0,309629.0,290859.0,325285.0,347705.0,346695.0,332304.0,326236.0,305170.0,280488.0,278107.0,280801.0,284001.0,283434.0,280984.0,270102.0,257432.0,267848.0,286799.0,285526.0
33,2013-02-03,4464,273743.0,15644.0,0.0,1273.0,3718.0,58723.0,0.0,3211.0,191068.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,2,3,2.96,-2.28,0.33,3.13,9.71,8.87,-0.84,3.30,69.88,0.02,97.48,9.45,12.35,...,0.0,0,1,34,0.552101,0.833777,0.920659,0.390368,0.718792,-0.695225,262321.0,266134.0,268874.0,285681.0,303496.0,303546.0,298866.0,309993.0,321288.0,313799.0,314414.0,306209.0,309629.0,290859.0,325285.0,347705.0,346695.0,332304.0,326236.0,305170.0,280488.0,278107.0,280801.0,284001.0,283434.0,280984.0,270102.0,257432.0,267848.0,286799.0
34,2013-02-04,4464,295369.0,28333.0,0.0,570.0,3381.0,78809.0,0.0,3134.0,181039.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,2,4,4.67,1.85,3.26,4.77,11.67,10.80,-0.88,4.52,82.94,0.00,98.21,7.31,10.12,...,0.0,0,1,35,0.566362,0.824157,0.933542,0.358468,0.669290,-0.743001,273743.0,262321.0,266134.0,268874.0,285681.0,303496.0,303546.0,298866.0,309993.0,321288.0,313799.0,314414.0,306209.0,309629.0,290859.0,325285.0,347705.0,346695.0,332304.0,326236.0,305170.0,280488.0,278107.0,280801.0,284001.0,283434.0,280984.0,270102.0,257432.0,267848.0
35,2013-02-05,4464,287424.0,38139.0,0.0,764.0,2067.0,72157.0,0.0,3162.0,171033.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,2,5,6.65,5.23,5.94,6.64,8.27,11.44,3.16,5.68,89.69,0.19,97.68,6.63,9.32,...,0.0,0,1,36,0.580455,0.814292,0.945320,0.326144,0.616621,-0.787260,295369.0,273743.0,262321.0,266134.0,268874.0,285681.0,303496.0,303546.0,298866.0,309993.0,321288.0,313799.0,314414.0,306209.0,309629.0,290859.0,325285.0,347705.0,346695.0,332304.0,326236.0,305170.0,280488.0,278107.0,280801.0,284001.0,283434.0,280984.0,270102.0,257432.0
36,2013-02-06,4464,294084.0,50421.0,0.0,816.0,2624.0,70043.0,0.0,3227.0,166848.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,2013,2,6,2.71,0.23,1.47,2.94,7.97,6.41,-1.56,4.03,84.50,0.10,96.65,6.42,8.96,...,0.0,0,1,37,0.594376,0.804187,0.955979,0.293434,0.561034,-0.827793,287424.0,295369.0,273743.0,262321.0,266134.0,268874.0,285681.0,303496.0,303546.0,298866.0,309993.0,321288.0,313799.0,314414.0,306209.0,309629.0,290859.0,325285.0,347705.0,346695.0,332304.0,326236.0,305170.0,280488.0,278107.0,280801.0,284001.0,283434.0,280984.0,270102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,2022-05-11,4464,218296.0,34225.0,0.0,1909.0,10139.0,40944.0,-306.0,3115.0,128276.0,0.0,0.0,0.0,0.0

## Fill the missing data

In [16]:
def replace_val(df,date1,date2): #on remplace les NaN dans l intervalle de date
    for col in df.columns:
        imputer = KNNImputer(n_neighbors=7) # Instantiate a SimpleImputer object with your strategy of choice
        imputer.fit(df[[col]]) # Call the "fit" method on the object
        df[col] = imputer.transform(df[[col]]) # Call the "transform" method on the object

In [17]:
df['Date']=pd.to_datetime(df['Date'])
df = df.set_index('Date').asfreq('D')
replace_val(df,'2022-04-30','2022-05-06')

### Select relevant features

In [18]:
df.drop(columns=['t - 1', 't - 2', 't - 3', 't - 4', 't - 5',
       't - 6', 't - 7', 't - 8', 't - 9', 't - 10', 't - 11', 't - 12',
       't - 13', 't - 14', 't - 15', 't - 16', 't - 17', 't - 18', 't - 19',
       't - 20', 't - 21', 't - 22', 't - 23', 't - 24', 't - 25', 't - 26',
       't - 27', 't - 28', 't - 29', 't - 30','sin365_1', 'cos365_1', 'sin365_2', 'cos365_2',
       'sin365_3', 'cos365_3','bank_holiday', 'school_holidays',
       'season','Thermique (MW)', 'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)',
       'Hydraulique (MW)', 'Pompage (MW)', 'Bioénergies (MW)',
       'Ech. physiques (MW)', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'TCO Thermique (%)',
       'TCH Thermique (%)', 'TCO Nucléaire (%)', 'TCH Nucléaire (%)',
       'TCO Eolien (%)', 'TCH Eolien (%)', 'TCO Solaire (%)',
       'TCH Solaire (%)', 'Column 30','Code INSEE région','sin_month', 'cos_month',
        'num_day',
        'YEAR', 'MONTH', 'DAY',
        'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE',
       'T2M_MAX', 'T2M_MIN','PS', 'QV2M','WS10M','WS50M','RH2M','week_day' #temoin
        #'PRECTOTCORR' # PRECTOTCORR is bias-corrected total precipitation
        #'sin_day','cos_day'
                ],inplace=True)

### Look of the dataset

In [19]:
df.head()

,Consommation (MW),T2M,PRECTOTCORR,sin_day,cos_day
Date,,,,,
2013-02-02,262321.0,5.55,1.94,-0.974928,-0.222521
2013-02-03,273743.0,2.96,0.02,-0.781831,0.623490
2013-02-04,295369.0,4.67,0.00,0.000000,1.000000
2013-02-05,287424.0,6.65,0.19,0.781831,0.623490
2013-02-06,294084.0,2.71,0.10,0.974928,-0.222521


# Compute energy consumption

## Fonction history (deep learning)

In [20]:
def plot_loss_mape(history):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13,4))
    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('Model loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    #ax1.set_ylim(ymin=0, ymax=10000)
    ax1.legend(['Train', 'Validation'], loc='best')
    ax1.grid(axis="x",linewidth=0.5)
    ax1.grid(axis="y",linewidth=0.5)    
    
    ax2.plot(history.history['mape'])
    ax2.plot(history.history['val_mape'])
    ax2.set_title('MAPE')
    ax2.set_ylabel('MAPE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylim(ymin=0, ymax=5)
    ax2.legend(['Train', 'Validation'], loc='best')
    ax2.grid(axis="x",linewidth=0.5)
    ax2.grid(axis="y",linewidth=0.5)    

    plt.show()

## Definition of the compute function

In [21]:
def compute_energy_forecast(df_vierge, Date_start_test, N_reg, model, methode='machine_learning', nb_epoch=100, batch_size=32):
    
    df_base = df_vierge.copy()
    
    Date_start_test = pd.to_datetime(Date_start_test)
    Date_fin_test =  Date_start_test + timedelta(days=N_test-1)
    
    if len(df_base.loc[df_base.index == Date_fin_test]) == 0:
        date_max=pd.to_datetime('2022-05-14')-timedelta(days=N_test-1)
        print(f'Erreur, veuillez saisir une date de début de test inférieure ou egale à : {str(date_max)}')
        return 0,0,0

    if len(df_base.loc[df_base.index == Date_start_test-timedelta(days=2*N_reg)]) == 0:
        date_limite_deb=pd.to_datetime('2013-03-04')+timedelta(days=2*N_reg)
        print(f'Erreur, veuillez saisir une date de début de test supérieure ou egale à : {str(date_limite_deb)}')
        return 0,0,0
    
    for i in range(1, N_reg+1):
        df_base[f't - {i}'] = df_base['Consommation (MW)'].shift(i)
    
    df_train = df_base.loc[(df_base.index < Date_start_test)]
    y_train = df_train['Consommation (MW)']
    df_test = df_base.loc[(df_base.index >= Date_start_test) & (df_base.index <= Date_fin_test)]
    y_true  = df_test['Consommation (MW)']
    df_train = df_train.copy()
    df_test = df_test.copy()
    df_val = df_test.copy()
    
    for i in range(1, N_reg+1):
        df_test[f't - {i}'] = float('NaN')

    #for i in range(1, N_reg+1):
    #    df_train[f't - {i}'] = df_train['Consommation (MW)'].shift(i)
    #    df_test[f't - {i}'] = float('NaN')
        
    X_train = df_train.drop(columns='Consommation (MW)')[N_reg:] #retrait des NaN
    y_train=df_train['Consommation (MW)'][N_reg:] #retrait des NaN
           
    pipe = make_pipeline(StandardScaler(), model)
    
    if methode == 'machine_learning':
        model = pipe.fit(X_train, y_train)
    elif methode == 'deep_learning':
        history = model.fit(X_train, y_train, epochs=nb_epochs, batch_size=batch_size, verbose=1,validation_split=0.3)
        plot_loss_mape(history)
    
    v_reg = pd.DataFrame(index=['0'])
    
    y_pred=[]

    for i in range(0,N_test):
        X_test=df_test.iloc[[i]].drop(columns='Consommation (MW)')
        if i==0:
            for j in range(1, N_reg+1):
                v_reg[f't - {j}'] = df_train['Consommation (MW)'][-j]
        for j in range(1, N_reg+1):
            X_test[f't - {j}'] = v_reg[f't - {j}'][0]
        out=model.predict(X_test)
        
        if methode == 'machine_learning':
            y_pred.append(out[0])
        elif methode == 'deep_learning':
            y_pred.append(out[0][0])
        
        if N_reg > 0:
            v_reg=v_reg.shift(1,axis=1)
            v_reg.iloc[0,0]=out[0]
    
    y_pred=pd.Series(y_pred,index=y_true.index)
    
    return y_true, y_pred, y_train

## Inputs

### Fenêtre de test et paramètres

ATTENTION, les conditions ci-dessous doivent impérativement être respectées :

Date_debut_test > 2013-02-02 et Date_debut_test <= 2022-05-15

Date_debut_test <= 2022-05-14 - N_test + 1

Date_debut_test >= 2013-03-04 + 2*N_reg

In [37]:
#model machine learning

Date_debut_test = '2020-06-01'    
N_test = 14
N_reg=1 #Nombre de jours auto-régressifs
methode = 'deep_learning'#  'machine_learning' #'deep_learning'
nb_epochs = 50 # si methode = deep_learning, conseil 150
batch_size = 8 # si methode = deep_learning

### Choix du model

#### Machine learning

In [38]:
#model = LinearRegression() # -
#model = DecisionTreeRegressor() ++
#model = XGBRegressor() ++
#model = RandomForestRegressor() ++
#model = BaggingRegressor() # +++
#model = AdaBoostRegressor() # ---
#model = GradientBoostingRegressor() # o
#model = KNeighborsRegressor() #-
#model = StackingRegressor(estimators=[('lr', RidgeCV()),('svr', LinearSVR(random_state=42))],final_estimator=RandomForestRegressor()) # ----
#model = SVR() # ---
#model = SGDRegressor() -

#### Deep learning

In [54]:
print('N_reg+df.shape[1]-1=',N_reg+df.shape[1]-1)
nb_feature = N_reg+df.shape[1]-1
model = Sequential()

# Input Layer
model.add(layers.Dense(nb_feature, input_dim=nb_feature, activation='relu')) #input_dim = nb_features dans X_train

# Hidden Layers
model.add(layers.Dense(nb_feature-1, activation='relu'))
model.add(layers.Dense(nb_feature-2, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

# Predictive Layer
model.add(layers.Dense(1, activation='linear'))

model.compile(loss='msle', optimizer=Adam(learning_rate = 0.0001),metrics = ['mape'])

N_reg+df.shape[1]-1= 5


In [55]:
df

,Consommation (MW),T2M,PRECTOTCORR,sin_day,cos_day
Date,,,,,
2013-02-02,262321.0,5.55,1.94,-0.974928,-0.222521
2013-02-03,273743.0,2.96,0.02,-0.781831,0.623490
2013-02-04,295369.0,4.67,0.00,0.000000,1.000000
2013-02-05,287424.0,6.65,0.19,0.781831,0.623490
2013-02-06,294084.0,2.71,0.10,0.974928,-0.222521
...,...,...,...,...,...
2022-05-11,218296.0,18.10,0.00,0.974928,-0.222521
2022-05-12,209770.0,18.13,0.00,0.433884,-0.900969
2022-05-13,201061.0,18.01,0.10,-0.433884,-0.900969


## Call of the forecast function

In [56]:
y_true, y_pred, y_train = compute_energy_forecast(df, Date_debut_test, N_reg, model, methode, nb_epochs, batch_size)

Epoch 1/50
234/234 [==============================] - 1s 2ms/step - loss: 151.8599 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 2/50
234/234 [==============================] - 0s 2ms/step - loss: 151.8599 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 3/50
234/234 [==============================] - 0s 2ms/step - loss: 151.8598 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 4/50
234/234 [==============================] - 0s 1ms/step - loss: 151.8598 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 5/50
234/234 [==============================] - 0s 1ms/step - loss: 151.8599 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 6/50
234/234 [==============================] - 0s 2ms/step - loss: 151.8598 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 7/50
234/234 [==============================] - 0s 1ms/step - loss: 151.8599 - mape: 100.0000 - val_loss: 150.6640 - val_mape: 100.0000
Epoch 

KeyboardInterrupt: 

## Display results for a model

In [ ]:
plt.figure(figsize=(15,11))
y_pred.plot(color = 'green')
#y_train.plot(color = 'blue')
y_true.plot(color = 'red')

MAPE_MODEL = mean_absolute_percentage_error(y_true, y_pred) # MAE en pourcentage 
y_baseline = np.ones(N_test)*y_train[-1]
MAPE_baseline = mean_absolute_percentage_error(y_true, y_baseline) # MAE en pourcentage
print(f'MAPE_MODEL = {MAPE_MODEL}, MAPE_baseline = {MAPE_baseline}')

## Cross-validation to select the best model

In [ ]:
model = RandomForestRegressor() #0.032
#model = GradientBoostingRegressor() #0.033
#model = BaggingRegressor() #0.035
#model = XGBRegressor() #0.035
#model = KNeighborsRegressor() #0.036
#model = DecisionTreeRegressor() #0.046
#model = AdaBoostRegressor() #0.050
#model = LinearRegression() #0.060
#model = SGDRegressor() #0.059
# model = baseline #0.066
#model = StackingRegressor(estimators=[('lr', RidgeCV()),('svr', LinearSVR(random_state=42))],final_estimator=RandomForestRegressor()) #0.121
#model = SVR() #0.133


N_test = 14
N_reg=1 #Nombre de jours auto-régressifs

dict_result_model=dict()
dict_result_baseline=dict()

for i in range(0,20):
    Date_debut_test = pd.to_datetime('2020-06-01') + timedelta(days=15)*i
    y_true, y_pred, y_train = compute_energy_forecast(df, Date_debut_test, N_reg,model)
    y_baseline = np.ones(N_test)*y_train[-1]
    dict_result_model[i] = mean_absolute_percentage_error(y_true, y_pred)
    dict_result_baseline[i] = mean_absolute_percentage_error(y_true, y_baseline)

In [17]:
mean = 0
i=0
for value in dict_result_model.values():
    i=i+1
    mean = mean +value
mean = mean/i
mean

0.03090810981406699

In [18]:
mean = 0
i=0
for value in dict_result_baseline.values():
    i=i+1
    mean = mean +value
mean = mean/i
mean

0.06592970433391471